In [1]:
import sys
sys.path.insert(0,'G:/dl-pytorch/2_object_detection/')

In [2]:
exec(open("Inference/inference.py").read())

<Figure size 720x720 with 0 Axes>

In [1]:
import cv2
from models.model import SSD
from lib  import *
from data.data_loader.transform import DataTransform
from data.data_loader.cfg import *

In [2]:
net = SSD(phase="inference", cfg=cfg)
net_weights = torch.load("G:/dl-pytorch/2_object_detection/data/weights/ssd300_100.pth", map_location={"cuda:0":"cpu"})
net.load_state_dict(net_weights)

<All keys matched successfully>

In [ ]:
cap = cv2.VideoCapture(0)
while True:
    ret, frame = cap.read()

    color_mean = (104, 117, 123)
    input_size = 300
    transform = DataTransform(input_size, color_mean)

    phase = "val"
    img_tranformed, boxes, labels = transform(frame, phase, "", "") 

    img_tensor = torch.from_numpy(img_tranformed[:,:,(2,1,0)]).permute(2,0,1)

    net.eval()
    input = img_tensor.unsqueeze(0) #(1, 3, 300, 300)
    output = net(input)

    plt.figure(figsize=(10, 10))
    colors = [(255,0,0), (0,255,0), (0,0,255)]
    font = cv2.FONT_HERSHEY_SIMPLEX

    detections = output.data #(1, 21, 200, 5) 5: score, cx, cy, w, h
    scale = torch.Tensor(frame.shape[1::-1]).repeat(2)

    for i in range(detections.size(1)):
        j = 0
        while detections[0, i, j, 0] >= 0.6:
            score = detections[0, i, j, 0]
            pt = (detections[0, i, j, 1:]*scale).cpu().numpy()
            cv2.rectangle(frame,
                          (int(pt[0]), int(pt[1])),
                          (int(pt[2]), int(pt[3])),
                          colors[i%3], 2
                          )
            display_text = "%s: %.2f"%(classes[i-1], score)
            cv2.putText(frame, display_text, (int(pt[0]), int(pt[1])),
                font, 0.5, (200, 69, 255), 1, cv2.LINE_AA)
            j += 1
    cv2.imshow('frame',frame)
    if cv2.waitKey(1) == ord('q'):
        break
cv2.destroyAllWindows()
cap.release()